# Génération des prédictions des modèles pour l'année 2025

## Modèles à comparer

- GNN sur DataSet barycentrique sans critère d'erreur informé par la physique
- GNN sur DataSet barycentrique avec critère d'erreur informé par la physique
- GNN sur DataSet planétaire sans critère d'erreur informé par la physique
- LSTM sur DataSet planétaire sans critère d'erreur informé par la physique
- LSTM sur DataSet planétaire avec critère d'erreur informé par la physique

## 1. Chargement des librairies

In [1]:
from model.GNN import GNN_NBody, InteractionNetwork
from data import solarSystemDataSet
import torch
from torch_geometric.data import Data
import torch_geometric.nn
import torch_geometric.inspector
import inspect
import _operator
import typing
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from joblib import load
# from google.colab import drive
# drive.mount('/content/drive')

c:\Repos\GIF-7005-Project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Définition des chemins des fichiers (à ajuster selon votre environnement local)

In [8]:
body_coordinates_and_velocities_dataset_path = "G:/Mon disque/GIF-7005-Project/data/body_coordinates_and_velocities_from_1749-12-31_to_2200-01-09.json"
planetary_centroid_coordinates_and_velocities_dataset_path = "G:/Mon disque/GIF-7005-Project/data/planetary_centroid_coordinates_and_velocities_from_1749-12-31_to_2200-01-09.json"

gnn_planetary_centroid_vanilla_model_file_path = "G:/Mon disque/GIF-7005-Project/models/GNN_Planetary_Centroid_Vanilla/best_model_weights.pth"
gnn_planetary_centroid_vanilla_scaler_file_path = "G:/Mon disque/GIF-7005-Project/models/GNN_Planetary_Centroid_Vanilla/scaler.joblib"

gnn_planetary_centroid_pi_model_file_path = "G:/Mon disque/GIF-7005-Project/models/GNN_Planetary_Centroid_PI/model.pth"
gnn_planetary_centroid_pi_scaler_file_path = "G:/Mon disque/GIF-7005-Project/models/GNN_Planetary_Centroid_PI/scaler.joblib"

gnn_body_coordinates_vanilla_model_file_path = "G:/Mon disque/GIF-7005-Project/models/GNN_Planet_Coordinates_Vanilla/model.pth"
gnn_body_coordinates_vanilla_scaler_file_path = "G:/Mon disque/GIF-7005-Project/models/GNN_Planet_Coordinates_Vanilla/scaler.joblib"

gnn_body_coordinates_pi_model_file_path = "G:/Mon disque/GIF-7005-Project/models/GNN_Planet_Coordinates_PI/model.pth"
gnn_body_coordinates_pi_scaler_file_path = "G:/Mon disque/GIF-7005-Project/models/GNN_Planet_Coordinates_PI/scaler.joblib"

## 3. Chargement des modèles

In [9]:
torch.serialization.add_safe_globals(
    [
        GNN_NBody, 
        torch.nn.modules.linear.Linear, 
        torch.nn.modules.container.ModuleList, 
        InteractionNetwork, 
        torch_geometric.nn.aggr.basic.SumAggregation, 
        torch.nn.modules.container.Sequential,
        torch.nn.modules.activation.ReLU,
        torch_geometric.inspector.Inspector,
        torch_geometric.inspector.Signature,
        torch_geometric.inspector.Parameter,
        inspect._empty,
        _operator.getitem,
        typing.OrderedDict,
        typing.Union,
        type,
        int
    ])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

gnn_planetary_centroid_vanilla_model = GNN_NBody(input_dim=7, model_dim=128).to(device)
gnn_planetary_centroid_vanilla_model.load_state_dict(torch.load(gnn_planetary_centroid_vanilla_model_file_path, map_location=torch.device('cpu')))
gnn_planetary_centroid_vanilla_model.eval()

gnn_planetary_centroid_pi_model: GNN_NBody = torch.load(gnn_planetary_centroid_pi_model_file_path, map_location=torch.device('cpu'))
gnn_planetary_centroid_pi_model.eval()
gnn_planetary_centroid_pi_model.to(device)

gnn_body_coordinates_vanilla_model: GNN_NBody = torch.load(gnn_body_coordinates_vanilla_model_file_path, map_location=torch.device('cpu'))
gnn_body_coordinates_vanilla_model.eval()
gnn_body_coordinates_vanilla_model.to(device)

gnn_body_coordinates_pi_model: GNN_NBody = torch.load(gnn_body_coordinates_pi_model_file_path, map_location=torch.device('cpu'))
gnn_body_coordinates_pi_model.eval()
gnn_body_coordinates_pi_model.to(device)

Using device: cpu


GNN_NBody(
  (node_encoder): Linear(in_features=7, out_features=128, bias=True)
  (interaction_layers): ModuleList(
    (0-2): 3 x InteractionNetwork()
  )
  (output_decoder): Linear(in_features=128, out_features=6, bias=True)
)

## 4. Chargement des scalers et des jeux de données

In [10]:
# Chargement des scalers spécifiques à chaque modèle.
gnn_planetary_centroid_vanilla_scaler: StandardScaler = load(gnn_planetary_centroid_vanilla_scaler_file_path)
gnn_planetary_centroid_pi_scaler: StandardScaler = load(gnn_planetary_centroid_pi_scaler_file_path)
gnn_body_coordinates_vanilla_scaler: StandardScaler = load(gnn_body_coordinates_vanilla_scaler_file_path)
gnn_body_coordinates_pi_scaler: StandardScaler = load(gnn_body_coordinates_pi_scaler_file_path)

# Chargement des données de référence dans un DataFrame.
df_body_coordinates: pd.DataFrame = pd.read_json(body_coordinates_and_velocities_dataset_path, lines=True)
df_planetery_centroid_coordinates: pd.DataFrame = pd.read_json(planetary_centroid_coordinates_and_velocities_dataset_path, lines=True)

c:\Repos\GIF-7005-Project\.venv\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Repos\GIF-7005-Project\.venv\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.5.2 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [11]:
display(df_body_coordinates.sample(5))
display(df_planetery_centroid_coordinates.sample(5))

,body_id,body_name,body_mass,datetime_jd,datetime_str,x,y,z,vx,vy,vz
445897,299,Venus,4.867500e+24,2477392.5,A.D. 2070-Oct-08 00:00:00.0000,-0.700744,-0.172892,0.037963,0.004685,-0.019740,-0.000544
368581,299,Venus,4.867500e+24,2400076.5,A.D. 1859-Feb-01 00:00:00.0000,-0.687736,0.200915,0.042532,-0.005961,-0.019448,0.000087
306397,199,Mercure,3.301100e+23,2502261.5,A.D. 2138-Nov-10 00:00:00.0000,0.035186,0.313578,0.021591,-0.033593,0.004358,0.003428
885093,599,Jupiter,1.898200e+27,2423481.5,A.D. 1923-Mar-03 00:00:00.0000,-4.133356,-3.505842,0.107003,0.004782,-0.005406,-0.000085
208486,199,Mercure,3.301100e+23,2404350.5,A.D. 1870-Oct-15 00:00:00.0000,0.073709,0.294345,0.017300,-0.032889,0.008023,0.003684


,body_id,body_name,body_mass,datetime_jd,datetime_str,x,y,z,vx,vy,vz
1023906,6,Saturne,5.684600e+26,2397925.5,A.D. 1853-Mar-13 00:00:00.0000,5.726422,7.103525,-0.352694,-0.004649,0.003489,0.000122
1310641,7,Uranus,8.683200e+25,2520291.5,A.D. 2188-Mar-22 00:00:00.0000,16.047141,11.617590,-0.164383,-0.002335,0.003001,0.000041
772196,4,Mars,6.417100e+23,2474953.5,A.D. 2064-Feb-03 00:00:00.0000,1.238862,0.726548,-0.014901,-0.006598,0.013248,0.000439
852767,5,Jupiter,1.898600e+27,2391155.5,A.D. 1834-Aug-30 00:00:00.0000,2.445651,4.384723,-0.072653,-0.006677,0.004037,0.000134
485048,2,Venus,4.867500e+24,2516543.5,A.D. 2177-Dec-17 00:00:00.0000,0.099511,-0.723702,-0.016117,0.019878,0.002892,-0.001103


## 5. Prédictions avec les différents modèles

La méthodologie pour la comparaison des modèles sera la suivante :
1. Le point de départ sera la position et la vélocité des planètes (ou du barycentre du système planétaire) le 1er janvier 2025.
2. À partir de ce point de départ, chaque modèle devra prédire la position et la vélocité de la planète ou du barycentre du système planétaire au jour suivant, et ainsi du suite, jusqu'à la fin de l'année.
3. Les positions prédites seront ensuite comparées aux positions "réelles", soient celles récupérées du système Horizons du Jet Propulsion Laboratory de la NASA et déjà présentes dans nos datasets.

In [13]:
DELTA_SCALER = 1000.0

def predict_2025_coordinates(X_dataset: solarSystemDataSet.SolarSystemDataset, model: GNN_NBody, scaler: StandardScaler) -> pd.DataFrame:
    df_predictions_2025: pd.DataFrame = pd.DataFrame()

    # On part l'inférence à partir des positions et vélocitées des planètes le 1er janvier 2025.
    current_pos_and_velocities = X_dataset.states[0]

    with torch.no_grad():
        for day_index in range(len(X_dataset.states)):
            # Prédiction des features pour l'ensemble des corps (delta de positions et de vélocités).
            predicted_deltas_normalized = model(Data(x=current_pos_and_velocities, edge_index=X_dataset.edge_index))

            # "predicted_deltas_normalized" est de shape (9, 6), mais on veut l'additionner aux valeurs de "current_pos_and_velocities", de shape (9, 7).
            # On ajoute donc une colonne de zéros pour que les shapes soient compatibles et pour pouvoir faire l'addition.
            current_pos_and_velocities += torch.tensor(np.append(predicted_deltas_normalized.numpy(), np.zeros((9, 1)), axis=1)) / DELTA_SCALER

            # On dénormalise les valeurs et on les ajoute au DataFrame des prédictions.
            predicted_pos_and_velocities_norm_numpy = current_pos_and_velocities.numpy()
            predicted_pos_and_velocities_denorm = scaler.inverse_transform(predicted_pos_and_velocities_norm_numpy)

            # On transforme les prédictions dénormalisées en DataFrame et on ajoute à celui-ci une colonne pour identifier le jour associé à la prédiction.
            df_day_predictions: pd.DataFrame = pd.DataFrame(predicted_pos_and_velocities_denorm)
            df_day_predictions.insert(loc=0, column="day", value=np.full((9, 1), day_index, dtype=int))

            df_predictions_2025 = pd.concat([df_predictions_2025, df_day_predictions], ignore_index=True)

    df_predictions_2025.columns = ['day', 'x', 'y', 'z', 'vx', 'vy', 'vz', 'body_mass']

    return df_predictions_2025

# On extrait les données de l'année 2025 dans de nouveaux DataFrames.
df_body_coordinates_targets_2025 = df_body_coordinates[(df_body_coordinates["datetime_str"].str.startswith("A.D. 2025-"))]
df_body_coordinates_targets_2025['body_mass'] = np.log10(df_body_coordinates_targets_2025['body_mass'])
df_planetery_centroid_coordinates_targets_2025 = df_planetery_centroid_coordinates[(df_planetery_centroid_coordinates["datetime_str"].str.startswith("A.D. 2025-"))]
df_planetery_centroid_coordinates_targets_2025['body_mass'] = np.log10(df_planetery_centroid_coordinates_targets_2025['body_mass'])

X_body_coordinates_vanilla_dataset: solarSystemDataSet.SolarSystemDataset = solarSystemDataSet.SolarSystemDataset(
    dataframe=df_body_coordinates_targets_2025, 
    scaler=gnn_body_coordinates_vanilla_scaler)

X_body_coordinates_pi_dataset: solarSystemDataSet.SolarSystemDataset = solarSystemDataSet.SolarSystemDataset(
    dataframe=df_body_coordinates_targets_2025, 
    scaler=gnn_body_coordinates_pi_scaler)

X_planetery_centroid_coordinates_vanilla_dataset: solarSystemDataSet.SolarSystemDataset = solarSystemDataSet.SolarSystemDataset(
    dataframe=df_planetery_centroid_coordinates_targets_2025, 
    scaler=gnn_planetary_centroid_vanilla_scaler)

X_planetery_centroid_coordinates_pi_dataset: solarSystemDataSet.SolarSystemDataset = solarSystemDataSet.SolarSystemDataset(
    dataframe=df_planetery_centroid_coordinates_targets_2025, 
    scaler=gnn_planetary_centroid_pi_scaler)

# Prédiction des coordonnées pour l'année 2025 avec chaque modèle.
df_predictions_body_coordinates_vanilla_2025: pd.DataFrame = predict_2025_coordinates(
    X_dataset=X_body_coordinates_vanilla_dataset,
    model=gnn_body_coordinates_vanilla_model,
    scaler=gnn_body_coordinates_vanilla_scaler)

df_predictions_body_coordinates_pi_2025: pd.DataFrame = predict_2025_coordinates(
    X_dataset=X_body_coordinates_pi_dataset,
    model=gnn_body_coordinates_pi_model,
    scaler=gnn_body_coordinates_pi_scaler)

df_predictions_planetery_centroid_coordinates_vanilla_2025: pd.DataFrame = predict_2025_coordinates(
    X_dataset=X_planetery_centroid_coordinates_vanilla_dataset,
    model=gnn_planetary_centroid_vanilla_model,
    scaler=gnn_planetary_centroid_vanilla_scaler)

df_predictions_planetery_centroid_coordinates_pi_2025: pd.DataFrame = predict_2025_coordinates(
    X_dataset=X_planetery_centroid_coordinates_pi_dataset,
    model=gnn_planetary_centroid_pi_model,
    scaler=gnn_planetary_centroid_pi_scaler)

# Sauvegarde des prédictions pour comparaison ultérieure.
df_predictions_body_coordinates_vanilla_2025.to_json('results/gnn_planet_coord_vanilla/predicted_planet_coordinates_and_velocities_from_2025-01-01_to_2025-12-31.json', orient='records', lines=True)
df_predictions_body_coordinates_pi_2025.to_json('results/gnn_planet_coord_pi/predicted_planet_coordinates_and_velocities_from_2025-01-01_to_2025-12-31.json', orient='records', lines=True)
df_predictions_planetery_centroid_coordinates_vanilla_2025.to_json('results/gnn_planetery_centroid_coord_vanilla/predicted_planetery_centroid_coordinates_and_velocities_from_2025-01-01_to_2025-12-31.json', orient='records', lines=True)
df_predictions_planetery_centroid_coordinates_pi_2025.to_json('results/gnn_planetery_centroid_coord_pi/predicted_planetery_centroid_coordinates_and_velocities_from_2025-01-01_to_2025-12-31.json', orient='records', lines=True)

C:\Users\alpha\AppData\Local\Temp\ipykernel_652\3212165875.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_body_coordinates_targets_2025['body_mass'] = np.log10(df_body_coordinates_targets_2025['body_mass'])
C:\Users\alpha\AppData\Local\Temp\ipykernel_652\3212165875.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_planetery_centroid_coordinates_targets_2025['body_mass'] = np.log10(df_planetery_centroid_coordinates_targets_2025['body_mass'])
Processing data groups: 100%|██████████| 365/365 [00